In [4]:
from datetime import datetime
date = 1014
print(f"统计于 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
from show_result import show_result
import os
for f in os.listdir(f"results/tests_{date}")[:3]:
    print("\033[32m> {}\033[0m".format(f))
    show_result(f"results/tests_{date}/{f}")
    print()

统计于 2024-10-15 15:18:34
> exp_1_baseline_mpnn_lstm

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[mpnn_lstm] 4.441	1.588	7.966	24.56	4.704	1.569	8.964	26.953	5.314	2.049	11.206	34.741
[    epoch] 475	52	145	222	453	322	172	378	359	57	440	317

> exp_2_dynst_gt_only



[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 5.436	1.486	7.437	31.749	4.894	1.463	8.08	39.035	6.637	1.879	11.299	28.035
[    epoch] 248	422	106	448	466	256	162	487	486	276	385	2

> exp_3_missing_nodes_no_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 5.539	1.46	7.476	24.212	5.62	1.676	7.992	33.099	6.161	1.867	11.784	28.034
[    epoch] 338	347	247	448	365	90	162	493	281	490	235	2



False

In [ ]:
raise NotImplementedError
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')
